In [ ]:
import pandas as pd

gsm_data = pd.read_csv('../data/preprocessed/gsm_data.csv')
gsm_data = gsm_data.sample(frac = 1)

**DONT RUN THE ABOVE AFTER RUNNING ONCE if running the below code more than 1 time** \
It ensures all types of questions have a fair chance of being answered by the LLM \
GSM-symbolic groups questions together based on template. This causes bias for the first few templates.  

In [ ]:
api_key = '..'

In [3]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

def ask_gpt(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "assistant", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )

    return completion.choices[0].message.content

In [4]:
import re

def extract_final_ans(response):
    answer = response.replace('*', '')
    answer = answer.replace("," ,'')
    answer = answer.lower().split('answer: ')

    try:
        final = re.findall(r"\d+", answer[-1])[0]
    except IndexError:
        final = None
    
    return final

### Collect 50 incorrect solutions first:

In [23]:
def collect_responses(df, mistakes):
    llm_response = []
    n = 0

    for idx, row in df.iterrows():
        if n==mistakes:
            return llm_response
        
        question = row['question']
        answer = row['answer']
        
        prompt = f"""
        Answer the following question. Please use Chain of Thought and number each step:
        {question}
        
        Have your final answer strictly at the end of your response in this form:
        Answer: 
        """
        
        response = ask_gpt(prompt)
        
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()
        
        if ref_final != final:
            n += 1
            llm_response.append({'question': question, 'ref_answer': answer, 'llm_answer': response, 'final_ans': final})
            
        print("final_ans: ", final)
        print("ref_ans: ", ref_final)
        print('iteration: ', n)
    return llm_response

### Collecting responses

In [ ]:
llm_gsm = collect_responses(gsm_data, 100) # gather more than 50 incorrct solutions to be able to ensure you have 50 valid samples after filtering

In [ ]:
save_directory = "../data/responses"
df_gsm= pd.DataFrame(llm_gsm)

df_gsm.to_csv(save_directory + '/gpt4o_responses.csv')

### Self correction process - exact error step and type

In [ ]:
import pandas as pd

df_responses = pd.read_csv('../data/responses/gpt4o_responses_annotated.csv')
df_responses = df_responses.head(50)

In [31]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_step = row['Error Step']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']

        correction_prompt = f"""
        Here are definitions of error types:
        - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
        - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
        - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
        - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
        - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
        - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
        
        The following solution of the given question is generated using the same model as you. Consider the question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in Step {int(error_step)} due to {error_type}.  Can you revise your solution and carefully solve it again?
        """

        response = ask_gpt(correction_prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [32]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  625
LLM initial answer:  246000
LLM self revised:  225
Ref answer:  26
LLM initial answer:  0
LLM self revised:  0
Ref answer:  448
LLM initial answer:  526
LLM self revised:  78
Ref answer:  25
LLM initial answer:  100
LLM self revised:  100
Ref answer:  45
LLM initial answer:  180
LLM self revised:  45
Ref answer:  80
LLM initial answer:  60
LLM self revised:  60
Ref answer:  396
LLM initial answer:  1364
LLM self revised:  1364
Ref answer:  474
LLM initial answer:  560
LLM self revised:  560
Ref answer:  84
LLM initial answer:  57
LLM self revised:  7
Ref answer:  26
LLM initial answer:  36
LLM self revised:  26
Ref answer:  48
LLM initial answer:  36
LLM self revised:  1
Ref answer:  1287
LLM initial answer:  13
LLM self revised:  13
Ref answer:  330
LLM initial answer:  1890
LLM self revised:  1
Ref answer:  48
LLM initial answer:  36
LLM self revised:  36
Ref answer:  8
LLM initial answer:  7
LLM self revised:  7
Ref answer:  84
LLM initial answer:  180
LLM self revi

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/gpt4o_selfcorrected.csv')

### Self correction process - general error location w/ error type

In [8]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']


        if row['Error Step'] <= (row['Total Steps']/2):
            prompt = f"""
            Here are definitions of error types:
            - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
            - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
            - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
            - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
            - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
            - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
            
            The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

            Question: {question}

            Solution: {solution}

            You made a mistake in the upper half of your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
            """
        else:
            prompt = f"""
            Here are definitions of error types:
            - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
            - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
            - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
            - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
            - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
            - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
            
            The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

            Question: {question}

            Solution: {solution}

            You made a mistake in the latter half of your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
            """

        response = ask_gpt(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [9]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  625
LLM initial answer:  246000
LLM self revised:  246000
Ref answer:  26
LLM initial answer:  0
LLM self revised:  0
Ref answer:  448
LLM initial answer:  526
LLM self revised:  214
Ref answer:  25
LLM initial answer:  100
LLM self revised:  25
Ref answer:  45
LLM initial answer:  180
LLM self revised:  45
Ref answer:  80
LLM initial answer:  60
LLM self revised:  60
Ref answer:  396
LLM initial answer:  1364
LLM self revised:  1
Ref answer:  474
LLM initial answer:  560
LLM self revised:  468
Ref answer:  84
LLM initial answer:  57
LLM self revised:  57
Ref answer:  26
LLM initial answer:  36
LLM self revised:  1
Ref answer:  48
LLM initial answer:  36
LLM self revised:  5
Ref answer:  1287
LLM initial answer:  13
LLM self revised:  1399
Ref answer:  330
LLM initial answer:  1890
LLM self revised:  1
Ref answer:  48
LLM initial answer:  36
LLM self revised:  36
Ref answer:  8
LLM initial answer:  7
LLM self revised:  7
Ref answer:  84
LLM initial answer:  180
LLM self re

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/gpt4o_selfcorrected_genloc.csv')

### Self correction process - with no error location

In [11]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        error_type = re.sub(r'^[\d.]+', '', row['Error Reason'].strip('()')).lstrip()
        answer = row['ref_answer']

        prompt = f"""
        Here are definitions of error types:
        - Incorrect value of variable cited or used: If the numerical value of a known (input or intermediate) variable is cited or used incorrectly.
        - Value of a known variable calculated again: If the value of an input or intermediate (determined mid solution) variable is calculated when it's value is already provided/ known.
        - Irrelevant/incorrect variable cited or used: If a variable in a solution is found to lie outside of the ideal variable-set of the solutioin (thereby making the solution innacurate).
        - Relevant variable missing: If any of the required variables in a solutioin for a target variable is missing (thereby making the solution innacurate).
        - Unit Inconsistency: If during the execution of an expression, two or more unit incompatible quantities are operated between.
        - Calculation Error: If the resultant numerical value of an operation or set of operations between numerical values is incorrect. 
        
        The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in your solution due to {error_type}.  Can you revise your solution and carefully solve it again?
        """

        response = ask_gpt(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [12]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  625
LLM initial answer:  246000
LLM self revised:  1
Ref answer:  26
LLM initial answer:  0
LLM self revised:  0
Ref answer:  448
LLM initial answer:  526
LLM self revised:  214
Ref answer:  25
LLM initial answer:  100
LLM self revised:  25
Ref answer:  45
LLM initial answer:  180
LLM self revised:  45
Ref answer:  80
LLM initial answer:  60
LLM self revised:  60
Ref answer:  396
LLM initial answer:  1364
LLM self revised:  1364
Ref answer:  474
LLM initial answer:  560
LLM self revised:  86
Ref answer:  84
LLM initial answer:  57
LLM self revised:  84
Ref answer:  26
LLM initial answer:  36
LLM self revised:  26
Ref answer:  48
LLM initial answer:  36
LLM self revised:  1
Ref answer:  1287
LLM initial answer:  13
LLM self revised:  13
Ref answer:  330
LLM initial answer:  1890
LLM self revised:  1
Ref answer:  48
LLM initial answer:  36
LLM self revised:  36
Ref answer:  8
LLM initial answer:  7
LLM self revised:  7
Ref answer:  84
LLM initial answer:  180
LLM self revise

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/gpt4o_selfcorrected_noloc.csv')

### Baseline: No error location or error type

In [14]:
import re

def self_correct(df):
    llm_response_corrected = []

    for idx, row in df.iterrows():
        question = row['question']
        solution = row['llm_answer']
        answer = row['ref_answer']

        prompt = f"""
        The following solution is generated using the same model as you. Consider the following question and solution that uses the Chain of Thought strategy with each step numbered.

        Question: {question}

        Solution: {solution}

        You made a mistake in your solution.  Can you revise your solution and carefully solve it again?
        """

        response = ask_gpt(prompt)
        final = extract_final_ans(response)

        ref_final = answer.splitlines()
        ref_final = ref_final[-1].replace("#",'')
        ref_final = ref_final.strip()

        print("Ref answer: ", ref_final)
        print("LLM initial answer: ", row['final_ans'])
        print("LLM self revised: ", final)
        
        llm_response_corrected.append({'question': question, 'ref_answer': answer, 'llm_initial_answer': row['llm_answer'],
                                        'initial_final_ans': row['final_ans'], 'llm_revised_answer': response,
                                        'revised_final_ans': final, 'error_type': row['Error Reason'], 
                                        'error_step': row['Error Step'], 'total_steps': row['Total Steps'],
                                        'correct': ref_final==final})
    
    return llm_response_corrected
        

In [15]:
llm_self_corrected = self_correct(df_responses)

Ref answer:  625
LLM initial answer:  246000
LLM self revised:  1
Ref answer:  26
LLM initial answer:  0
LLM self revised:  0
Ref answer:  448
LLM initial answer:  526
LLM self revised:  78
Ref answer:  25
LLM initial answer:  100
LLM self revised:  100
Ref answer:  45
LLM initial answer:  180
LLM self revised:  45
Ref answer:  80
LLM initial answer:  60
LLM self revised:  60
Ref answer:  396
LLM initial answer:  1364
LLM self revised:  1364
Ref answer:  474
LLM initial answer:  560
LLM self revised:  560
Ref answer:  84
LLM initial answer:  57
LLM self revised:  1
Ref answer:  26
LLM initial answer:  36
LLM self revised:  36
Ref answer:  48
LLM initial answer:  36
LLM self revised:  48
Ref answer:  1287
LLM initial answer:  13
LLM self revised:  13
Ref answer:  330
LLM initial answer:  1890
LLM self revised:  1
Ref answer:  48
LLM initial answer:  36
LLM self revised:  1
Ref answer:  8
LLM initial answer:  7
LLM self revised:  7
Ref answer:  84
LLM initial answer:  180
LLM self revise

In [ ]:
save_directory = "../data/responses"

df_corrected = pd.DataFrame(llm_self_corrected)
df_corrected.to_csv(save_directory + '/gpt4o_selfcorrected_baseline.csv')